In [1]:
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [2]:
#data importing
arch=np.load('data/month_year_train.npz')
print ('keys:',list(arch.keys()))
from sklearn.model_selection import train_test_split
train_x=arch['x']
train_y=arch['y']
x,eva_x,y,eva_y=train_test_split(train_x,train_y,test_size=0.3,random_state=42)

keys: ['x', 'y']


In [3]:
import lightgbm as lgb
train_data=lgb.Dataset(x,label=y)
validation_data=lgb.Dataset(eva_x,label=eva_y,reference=train_data)

In [4]:
param={'num_leaves':31,'objective':'regression'}
param['metric']='rmse'

In [10]:
num_round=1000
bst=lgb.train(param,train_data,num_round,valid_sets=[validation_data],early_stopping_rounds=10)

[1]	valid_0's rmse: 717.075
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's rmse: 657.07
[3]	valid_0's rmse: 603.776
[4]	valid_0's rmse: 556.965
[5]	valid_0's rmse: 515.22
[6]	valid_0's rmse: 477.292
[7]	valid_0's rmse: 443.288
[8]	valid_0's rmse: 414.359
[9]	valid_0's rmse: 388.592
[10]	valid_0's rmse: 364.81
[11]	valid_0's rmse: 345.369
[12]	valid_0's rmse: 327.114
[13]	valid_0's rmse: 312.442
[14]	valid_0's rmse: 298.648
[15]	valid_0's rmse: 286.797
[16]	valid_0's rmse: 277.474
[17]	valid_0's rmse: 268.729
[18]	valid_0's rmse: 261.149
[19]	valid_0's rmse: 255.071
[20]	valid_0's rmse: 249.801
[21]	valid_0's rmse: 244.578
[22]	valid_0's rmse: 240.604
[23]	valid_0's rmse: 237.253
[24]	valid_0's rmse: 234.335
[25]	valid_0's rmse: 231.447
[26]	valid_0's rmse: 229.112
[27]	valid_0's rmse: 226.687
[28]	valid_0's rmse: 224.751
[29]	valid_0's rmse: 223.249
[30]	valid_0's rmse: 221.311
[31]	valid_0's rmse: 219.814
[32]	valid_0's rmse: 218.341
[33]	valid_0's rmse: 2

In [12]:
y_pred=np.rint(bst.predict(x))

In [13]:
def mean_metrics(y_pred,y):
    n_samples=y_pred.shape[0]
    y_mean=np.mean(y)
    return (1-((((y-y_pred)**2).sum()/n_samples)**0.5)/y_mean)

In [14]:
mean_metrics(y_pred,y)

0.8227381495934718

In [16]:
eva_y_pred=np.rint(bst.predict(eva_x))
mean_metrics(eva_y_pred,eva_y)

0.706001431234099

In [18]:
bst.save_model('joblib/gbdt_model.txt')